## Search

Beispiel 1:

    ##########
    #   #    #
    #   ##   #
    # #Z#  ###
    # ## #   #
    #   S #  #
    #     #  #
    #        #
    ##########


Wir sind in einem Labyrinth an Position S. Wir haben können eine Schritt horizontal oder vertikal machen, sofern wir nicht an eine Wand ('#') stoßen. Unser Ziel ist es, auf kürzestem Weg zur Zielposition Z zu gelangen. 


Beispiel 2:

Wir möchten durch Verschieben der Kacheln im 8Puzzle den Zielzustand erreichen.

<img src='such_bild1.png' width='400'>

### Search Space


In beiden Fälle beginnen wir mit einem Anfangszustand (startstate). Von einem Zustand können wir durch erlaubte Aktionen in verschiedene Folgezustände gelangen. Wir stellen uns dies als einen Graphen vor, die Zustände sind die Knoten und die Kanten die erlaubten Übergänge zwischen zwei Zuständen. Wir nennen das zu durchsuchende Gebilde *Search Space*.

<img src='search1.png' width='400'>

Aus der Ferne stellen wir uns den *Search Space* wie einen breiter werdenden Raum vor, in dem wir einen Pfad (=eine Folge von Aktionen) zum Ziel suchen.

<img src='search2.png' width='500'>

Wir können den Search Space in drei Teile unterteilen. Einen Teil mit den Zuständen, die wir schon daraufhin untersucht haben, ob sie Ziel sind (*explored*). Einen Teil mit Zuständen, die wir als Nachbarzustände schon gesehen haben, aber noch untersuchen müssen (*frontier*). Und einen unbekannten Raum von Zuständen, die wir noch nicht gesehen haben (*unknown*).

<img src='search3.png' width='600'>

Die einzelnen Algorithmen unterscheiden sich dadurch, welche Zustände sie als nächstes zur Überprüfung aus der frontier entnehmen.

In unseren Implementationen nutzen wir statt der *explored*-Menge  ein dictionary *prev*, in dem wir uns die Vorgänger von allen Zuständen merken, die uns begegnen. *prev* umfasst die beiden Bereiche *explored* und *frontier*. Wenn wir am Ziel angekommen sind, können wir mittels *prev* den Weg zurück zum Start rekonstruieren.

### Breitensuche (bfs = breadth first search)

Die *frontier* ist eine Schlange (queue). Wenn wir etwas zur Schlange hinzufügen (*append*), wird es hinten angehängt. Wenn wir etwas herausholen,
wird es vorne weggenommen (*popleft*). 

<img src='bfs.png' width='700'>

Bei der Breitensuche schieben wir die frontier horizontal ins unbekannt Gebiet weiter.
<img src='bfs1.png'>

In [22]:
from collections import deque
def bfs(startstate):
    ''' 
    returns: Tuple (prev, state) 
        prev: dictionary mit den Vorgängern der untersuchten Spielstellungen,            
        state: Spielstellung, die den goaltest besteht
        wenn Ziel nicht gefunden: None, None
    '''   
    frontier =  deque([startstate])
    prev = {startstate:None}
    while frontier:
        state = frontier.popleft()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def getMoves(path):
    '''
    returns: Beschreibung des Pfads als eine Folge von Aktionen (Moves)
    '''
    moves = ''
    for i in range(len(path)-1):
        moves+=getMove(path[i],path[i+1])
    return moves

def nextstates(state):
    '''
    returns:  Liste mit möglichen Folgestellungen zu state
    '''
    pass

def goaltest(state):
    '''
    returns: True, wenn state eine Lösung ist
    '''
    pass

def getMove(s1, s2):
    '''
    returns: die Beschreibung des Übergangs von state s1 zu state s2
    '''
    pass
   
#Aufruf:
#prev, goal = bfs(startstate)
#path = reconstructPath(prev,goal) 
#print(getMoves(path))

#### Beispiel 1: Maze

Wir modellieren einen Zustand als eine Koordinate (x,y) mit (0,0) an der linken oberen Ecke. Die erste Koordinate gibt den Index der Zeile, die zweite den Index der Spalte an.

In [50]:
%%writefile maze1.txt
##########
#   #    #
#   ##   #
# #Z#  ###
# ## #   #
#   S #  #
#     #  #
#        #
##########

Overwriting maze1.txt


In [55]:
f = open('maze1.txt')       
grid = f.read().splitlines()  
f.close()
grid

['##########',
 '#   #    #',
 '#   ##   #',
 '# #Z#  ###',
 '# ## #   #',
 '#   S #  #',
 '#     #  #',
 '#        #',
 '##########']

In [ ]:
height = len(grid)
width = len(grid[0])
for x in range(height):
    for y in range(width):
        if grid[x][y] == 'S':
            start = (x,y)
        elif grid[x][y] == 'Z':
            ziel = (x,y)

print(f'Höhe = {height}, Breite = {width}')
print(f'{start=}, {ziel=}')

dirs = [(0,-1),(0,1),(-1,0),(1,0)]   

Höhe = 9, Breite = 10
start=(5, 4), ziel=(3, 3)


In [52]:
def nextstates(state):
    x, y = state
    tmp = []
    for xd, yd in dirs:
        x1 = x + xd
        y1 = y + yd
        if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != '#':
            tmp.append((x1,y1))
    return tmp    
    
def goaltest(state):
    return state == ziel

def getMove(s1, s2):
    x1,y1 = s1
    x2,y2 = s2
    if x1 < x2: return 'D'   # down
    if x1 > x2: return 'U'   # up
    if y1 < y2: return 'R'   # right
    if y1 > y2: return 'L'   # left

In [56]:
prev, goal = bfs(start)
path = reconstructPath(prev,goal) 
print(path)
print(f'Anzahl Aktionen = {len(path)-1}') 
print(getMoves(path))

[(5, 4), (5, 3), (5, 2), (5, 1), (4, 1), (3, 1), (2, 1), (2, 2), (2, 3), (3, 3)]
Anzahl Aktionen = 9
LLLUUURRD


#### Beispiel 2: 8Puzzle

<img src='such_bild1.png' width='400'>

Wir modellieren eine Spielstellung mit einem Tupel. Wir beschreiben die Aktionen durch die Bewegung der Leerstelle.

In [58]:
start = (7,2,4,5,0,6,8,3,1)
ziel  = (0,1,2,3,4,5,6,7,8)

In [59]:
def nextstates(state):
    tmp = []
    i = state.index(0)
    for d in [-3,3,-1,1]:
        j = i+d
        if 0 <= j < 9 and (i%3==j%3 or i//3==j//3):  # bleibt auf gleicher Zeile oder Spalte
            a = list(state)
            a[i],a[j] = a[j],a[i]
            tmp.append(tuple(a))
    return tmp  
    
def goaltest(state):
    '''
    returns: True, wenn state eine Lösung ist
    '''
    return state == ziel

def getMove(s1, s2):
    i1 = s1.index(0)   # Position der 0 in state s1  
    i2 = s2.index(0)   # Position der 0 in state s2    
    if i2 == i1-1: return 'L'   #left
    if i2 == i1+1: return 'R'   #right
    if i2 == i1+3: return 'D'   #down
    if i2 == i1-3: return 'U'   #up

In [60]:
prev, goal = bfs(start)
path = reconstructPath(prev,goal) 
print(path)
print(f'Anzahl Aktionen = {len(path)-1}') 
print(getMoves(path))

[(7, 2, 4, 5, 0, 6, 8, 3, 1), (7, 2, 4, 0, 5, 6, 8, 3, 1), (0, 2, 4, 7, 5, 6, 8, 3, 1), (2, 0, 4, 7, 5, 6, 8, 3, 1), (2, 5, 4, 7, 0, 6, 8, 3, 1), (2, 5, 4, 7, 3, 6, 8, 0, 1), (2, 5, 4, 7, 3, 6, 0, 8, 1), (2, 5, 4, 0, 3, 6, 7, 8, 1), (2, 5, 4, 3, 0, 6, 7, 8, 1), (2, 5, 4, 3, 6, 0, 7, 8, 1), (2, 5, 0, 3, 6, 4, 7, 8, 1), (2, 0, 5, 3, 6, 4, 7, 8, 1), (0, 2, 5, 3, 6, 4, 7, 8, 1), (3, 2, 5, 0, 6, 4, 7, 8, 1), (3, 2, 5, 6, 0, 4, 7, 8, 1), (3, 2, 5, 6, 4, 0, 7, 8, 1), (3, 2, 5, 6, 4, 1, 7, 8, 0), (3, 2, 5, 6, 4, 1, 7, 0, 8), (3, 2, 5, 6, 0, 1, 7, 4, 8), (3, 2, 5, 6, 1, 0, 7, 4, 8), (3, 2, 0, 6, 1, 5, 7, 4, 8), (3, 0, 2, 6, 1, 5, 7, 4, 8), (3, 1, 2, 6, 0, 5, 7, 4, 8), (3, 1, 2, 6, 4, 5, 7, 0, 8), (3, 1, 2, 6, 4, 5, 0, 7, 8), (3, 1, 2, 0, 4, 5, 6, 7, 8), (0, 1, 2, 3, 4, 5, 6, 7, 8)]
Anzahl Aktionen = 26
LURDDLURRULLDRRDLURULDDLUU
